## Use of Azure OpenAI, LangChain and Milvus
In this notebook, you'll vectorise content of your PDF file, upload it into Milvus vector store and then use Azure OpenAI endpoints and LangChain to find answers on the content-related questions.

### Setup
Define relevant Azure OpenAI and Milvus parameters.

In [1]:
# Import required packages
import openai
import os

# Define Azure OpenAI endpoint parameters
openai.api_type = "azure"
openai.api_version = os.getenv("OPENAI_API_VERSION") # Set AOAI API version value as env variable
openai.api_base = os.getenv("OPENAI_API_BASE") # Set AOAI endpoint value as env variable
openai.api_key = os.getenv("OPENAI_API_KEY") # Set AOAI API key as env variable
GPT_deployment = os.getenv("OPENAI_API_DEPLOY") # Set AOAI GPT deployment name as env variable
ADA_deployment = os.getenv("OPENAI_API_DEPLOY_EMBED") # Set AOAI Ada deployment name as env variable

# Define Milvus vector store parameters
MILVUS_HOST = "localhost"
MILVUS_PORT = "19530"

### Loading data
PDF document will be uploaded into array of documents, each document representing specific page.

In [2]:
from langchain.document_loaders import PyPDFLoader

# Create array of pages
loader = PyPDFLoader("data/NorthwindHealthPlus_BenefitsDetails.pdf")
pages = loader.load_and_split()

In [3]:
# Check number of pages
print(f"Number of pages in a new array: {len(pages)}")
print(f"Content of 1st page: {pages[0]}")

Number of pages in a new array: 109
Content of 1st page: page_content='Contoso Electronics  \nNorthwind Health Plus Plan' metadata={'source': 'data/NorthwindHealthPlus_BenefitsDetails.pdf', 'page': 0}


### Embedding data
Uisng text-embedding-ada-002 model to embed PDF pages.

Please, note that "chunk_size" in Python implementation actually refers to "batchSize" as correctly named in JavaScript implementation.
- https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.openai.OpenAIEmbeddings.html
- https://js.langchain.com/docs/api/embeddings_openai/interfaces/OpenAIEmbeddingsParams#batchsize

Currently text-embedding-ada-002 supports up to 16 embeddings per batch.
- https://learn.microsoft.com/en-us/azure/ai-services/openai/faq

In [4]:
from langchain.embeddings import OpenAIEmbeddings

# Initiating Ada embeddings
embeddings = OpenAIEmbeddings(engine=ADA_deployment, chunk_size=16);

c:\Users\lturakulov\AppData\Local\Programs\Python\Python38\lib\site-packages\langchain\embeddings\openai.py:214: UserWarning: WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.
  warnings.warn(


In [5]:
# Testing embeddings
text = "This is a test document."
query_result = embeddings.embed_query(text)
print(query_result)

[-0.0031584086849463185, 0.011094410093459926, -0.0040013172535311795, -0.011747414319539315, -0.001015321743093967, 0.010781235205489612, -0.010368109854788978, -0.005297331218210429, -0.00988168849149678, -0.02616015258102869, 0.020376401396509824, 0.02257529295012674, -0.0075228762906436, 0.01728462487878975, -0.006003642016261069, 0.01912369811933854, 0.021255957272898585, -0.01564545070105659, 0.007669469247149229, -0.018364081680639154, -0.0006909018925560501, -0.006416766901300451, -0.01098113426830524, 0.017950956329938518, -0.022135515011932363, -0.003076783156686395, 0.014379422061460938, -0.029984892949954402, 0.018524000930891654, -0.007916011269401418, 0.01006826032924303, -0.019456863844912922, -0.003688141899899612, -0.02424111971535365, -0.005530546946715746, 0.003824739670433833, -0.005653818190672467, -0.02945182816156439, 0.018603960556017908, -0.01661829342764098, 0.00752953990317829, 0.012420409383222773, -0.0009903343602420136, -0.010721265486644923, 0.01000162792

Converting text into vector mebeddings and storing them in Milvus vector store.

In [6]:
from langchain.vectorstores import Milvus

# Creating new collection in Milvus vector store
vector_store = Milvus.from_documents(
    pages,
    embedding=embeddings,
    collection_name="northwindhealthplus_Nov2023",
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT}
)

In [7]:
# Retrieving stored collection from Milvus vector store
vector_store = Milvus(
    embedding_function=embeddings,    
    collection_name="northwindhealthplus_Nov2023",
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT}
)

### Question answering
Findings answers from the content in Milvus.

In [8]:
# Test query
query = "What is Northwind Health Plus?"

#### 1. Similarity search
Finding the most relevant page from the vector store.

In [9]:
# Similarity search
docs = vector_store.similarity_search(query)

print(f"Original source document: {docs[0].metadata['source']}")
print(f"Original source document page: {docs[0].metadata['page']}")
print("--------------------------------------")
print(docs[0].page_content)


Original source document: data/NorthwindHealthPlus_BenefitsDetails.pdf
Original source document page: 93
--------------------------------------
The Northwind Health Plus plan is a group health plan that is sponsored by Contoso and 
administered by Northwind Health. As a participant in this group plan, you will have access 
to a wide range of health benefits and services.  
Your employer, Contoso, pays a portion of the premium for the plan, and you are 
responsible for paying the remaining portion. This will be deducted directly from your 
paycheck each month. In addition to the premium, you may be responsible for ce rtain costs 
when you receive health care services.  
Your contributions to the plan are based on the type of coverage you choose. For example, if 
you choose a single plan, you will pay a lower premium than if you choose a family plan. The 
premium and other costs you may incur when you receive health care services may change 
from year to year.  
It is important to note t

#### 2. Chaining with Map-Reduce
LangChain provides four chain types for question-answering:
- stuff,
- map_reduce,
- refine,
- map-rerank.

In [10]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import AzureOpenAI

# MapReduce chain
chain = load_qa_with_sources_chain(
    llm=AzureOpenAI(engine=GPT_deployment),
    chain_type="map_reduce",
    return_intermediate_steps=True
)
response = chain(
    {"input_documents": docs, "question": query},
    return_only_outputs=True
)


c:\Users\lturakulov\AppData\Local\Programs\Python\Python38\lib\site-packages\langchain\utils\utils.py:149: UserWarning: WARNING! engine is not default parameter.
                engine was transferred to model_kwargs.
                Please confirm that engine is what you intended.
  warnings.warn(


In [11]:
# Dialog results
print(f"Question: {query}")
print(f"Answer: {response['output_text']}")

Question: What is Northwind Health Plus?
Answer:  Northwind Health Plus is a comprehensive group health plan sponsored by Contoso and administered by Northwind Health. It provides coverage for medical, vision, and dental services, as well as prescription drugs, mental health and substance abuse services, and preventive care. It also offers a variety of in-network providers and coverage for emergency services. 
SOURCES: data/NorthwindHealthPlus_BenefitsDetails.pdf


#### 3. Retrieval-augmented generation (RAG)

In [12]:
from langchain.chains import RetrievalQA

# RAG chain
rag_chain = RetrievalQA.from_chain_type(
    llm=AzureOpenAI(engine=GPT_deployment),
    retriever=vector_store.as_retriever()
)
rag_response = rag_chain.run(query)

In [13]:
# Dialog results
print(f"Question: {query}")
print(f"Answer: {rag_response}")

Question: What is Northwind Health Plus?
Answer:  Northwind Health Plus is a comprehensive group health plan that is sponsored by Contoso and administered by Northwind Health. It provides coverage for medical, vision, and dental services, as well as prescription drug coverage, mental health and substance abuse coverage, and coverage for preventive care services.
